In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from digen import Benchmark


/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digens = pickle.load(f)

print(digens.keys())

digenkeys = list(digens.keys())

benchmark=Benchmark()
benchmodels = benchmark.get_models()
print(benchmodels)
#print(digenkeys)
print(digens['digen8_4426'])
digennoise = {}
traincsv = {}
testcsv = {}
train_best_preformance = {}
test_final_preformance ={}
lossfxn = {}

dict_keys(['digen8_4426', 'digen32_5191', 'digen39_5578', 'digen36_466', 'digen13_769', 'digen35_4426', 'digen1_6265', 'digen10_8322', 'digen23_5191', 'digen22_2433', 'digen24_2433', 'digen3_769', 'digen17_6949', 'digen26_7270', 'digen7_6949', 'digen16_5390', 'digen27_860', 'digen37_769', 'digen12_8322', 'digen6_466', 'digen25_2433', 'digen34_769', 'digen31_2433', 'digen33_769', 'digen38_4426', 'digen28_769', 'digen40_5390', 'digen2_6949', 'digen4_860', 'digen14_769', 'digen29_8322', 'digen21_6265', 'digen15_5311', 'digen19_7270', 'digen5_6949', 'digen11_7270', 'digen9_7270', 'digen20_5191', 'digen18_5578', 'digen30_4426'])
{'digen8_4426': 'safeDiv(X0, mul(X5, X2))', 'digen32_5191': 'mul(X3, mul(X1, min(X9, X6)))', 'digen39_5578': 'lt(mul(X7, X9), mul(X4, X0))', 'digen36_466': 'ne(lt(X5, X3), gt(X6, X8))', 'digen13_769': 'le(add(mul(X7, X9), mul(1.0, mul(X0, X8))), safeDiv(gt(min(sub(X1, X7), mul(safeDiv(X4, X9), sub(X1, X3))), le(X2, safeDiv(X5, eq(X8, X8)))), X0))', 'digen35_4426': '

In [3]:
for i in digenkeys:
    #print(i) 
    data = pd.DataFrame(digens[i]['X'])
    yval = pd.DataFrame(digens[i]['y'])
    yval.columns = ['y']
    yval.rename(columns = {'0':'y'}, inplace = True)
    noisy = pd.concat([data, yval], axis=1)
    noisy.index.name = 'InstanceId'

    ynonoise = pd.DataFrame(digens[i]['y_no_noise'])
    ynonoise.columns = ['y']
    ynonoise.rename(columns = {'0':'y'}, inplace = True)
    no_noise = pd.concat([data, ynonoise], axis=1)
    no_noise.index.name = 'InstanceId'
    
    droplist = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
    ground_truth = benchmodels[i]
    #print(ground_truth)
    for x in droplist:
        #print(x)
        if x not in ground_truth:
            #print('Drop')
            noisy = noisy.drop(x, axis=1)
            no_noise = no_noise.drop(x, axis=1)
            
    cols_list = no_noise.select_dtypes('number').columns.tolist()
    cols_list.remove('y')

    n_five = noisy.copy()
    n_ten = noisy.copy()
    n_twentyfive = noisy.copy()
    n_fifty = noisy.copy()

    no_five = no_noise.copy()
    no_ten = no_noise.copy()
    no_twentyfive = no_noise.copy()
    no_fifty = no_noise.copy()

    for x in noisy[cols_list]:
            n_five.loc[n_five.sample(frac=0.05).index, x] = np.nan
            n_ten.loc[n_ten.sample(frac=0.1).index, x] = np.nan
            n_twentyfive.loc[n_twentyfive.sample(frac=0.25).index, x] = np.nan
            n_fifty.loc[n_fifty.sample(frac=0.5).index, x] = np.nan

    for x in no_noise[cols_list]:
            no_five.loc[no_five.sample(frac=0.05).index, x] = np.nan
            no_ten.loc[no_ten.sample(frac=0.1).index, x] = np.nan
            no_twentyfive.loc[no_twentyfive.sample(frac=0.25).index, x] = np.nan
            no_fifty.loc[no_fifty.sample(frac=0.5).index, x] = np.nan

    cols_list = no_noise.select_dtypes('number').columns.tolist()
    cols_list.remove('y')
    
    outstring = './data/DigenStudy/No_Noise/' + i + '/' + i + 'no_missing.csv'
    outstring5 = './data/DigenStudy/No_Noise/' + i + '/' + i + 'no_5.csv'
    outstring10 = './data/DigenStudy/No_Noise/' + i + '/' + i + 'no_10.csv'
    outstring25 = './data/DigenStudy/No_Noise/' + i + '/' + i + 'no_25.csv'
    outstring50 = './data/DigenStudy/No_Noise/' + i + '/' + i + 'no_50.csv'
    try: 
        os.mkdir('./data/DigenStudy/No_Noise/' + i + '/')
    except OSError as error: 
        pass

    #tempstring = './data/testfolder/' + i + '.csv'
    no_noise.to_csv(outstring)
    no_five.to_csv(outstring5)
    no_ten.to_csv(outstring10)
    no_twentyfive.to_csv(outstring25)
    no_fifty.to_csv(outstring50)

    

    #no_noise.to_csv(tempstring)
    current_noise = digens[i]['noise']
    digennoise[i] = current_noise

    if current_noise != 0.0: 
        outstring = './data/DigenStudy/Noisy/' + i + '/' + i + 'n_missing.csv'
        outstring5 = './data/DigenStudy/Noisy/' + i + '/' + i + 'n_5.csv'
        outstring10 = './data/DigenStudy/Noisy/' + i + '/' + i + 'n_10.csv'
        outstring25 = './data/DigenStudy/Noisy/' + i + '/' + i + 'n_25.csv'
        outstring50 = './data/DigenStudy/Noisy/' + i + '/' + i + 'n_50.csv'
        #tempstring = './data/testfolder/' + i + '.csv'
        try: 
            os.mkdir('./data/DigenStudy/Noisy/' + i + '/')
        except OSError as error: 
            pass
        noisy.to_csv(outstring)
        n_five.to_csv(outstring5)
        n_ten.to_csv(outstring10)
        n_twentyfive.to_csv(outstring25)
        n_fifty.to_csv(outstring50)
        #noisy.to_csv(tempstring)

    '''
    X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

    train_set = pd.concat([X_train, y_train], axis =1)
    train_set.index.name = 'InstanceId'
    tempstring = './data/testfolder/' + i +'_train.csv'
    train_set.to_csv(tempstring)
    cleanrun = OptimizeClean(dataset_name=[i], n_trials=100, data_path='./data/testfolder')
    cleanrun.run()
    optimal = cleanrun.param
    current_auc = cleanrun.goal

    tempstring = './data/testfolder/' + i +'_train.csv'
    test_set = pd.concat([X_test, y_test], axis =1)
    test_set.index.name = 'InstanceId'

    try:
        most_recent_run = AutoRunner(**optimal, dataset_names=[i],data_path='./data/testfolder',
                                        gen_report=False, clean=False, class_label='target', 
                                        instance_label='InstanceID', ml_algorithms=["NB", "LR", "DT", "EN", "XGB", "LGB", "CGB", "SVM","GB", "RF"], 
                                        exclude=["GB", "RF","ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
        output_csv = most_recent_run.run(run_para=False)
        performance = pd.read_csv(output_csv)
        summary_chart = performance
        goal = performance["ROC AUC"].max()
        best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
        png_out = output_csv.removesuffix('Summary_performance_mean.csv')
        png_out = png_out + 'Summary_ROC.png'
        final_model_comparison = plt.show(png_out)
    except:
        print('EXCEPTION')
        
    print(current_auc-goal)
        '''


    



In [4]:
#Noisy checks:
noisy.head()
print(cols_list)


['X0', 'X1', 'X4', 'X6']
